In [1]:
import time
import random
import gc

# Set seed.
random.seed(a=100)

# Create our default list.
short_list = list(random.sample(range(1000000), 10))
long_list = list(random.sample(range(1000000), 10000))

## Quicksort
Quicksort is an old and reliable sorting algorithm. It is similar to merge sort in that it is a "divide and conquer" approach. It relies on partitioning the data around a value called the "pivot." For simplicity, this value is often taken to be the last (i=-1) element in the array. The partitioning function rearranges the values in the array such that all the valuse on the left are less than the pivot and all on the right are greater by iterating over the values and swapping where necessary. It finally swaps the value to the right of the partition with the pivot at the i=-1 position to achieve the required arrangement. This function edits the array in place, and returns the index where the pivot ends up.<br><br>
The quicksort function takes an array and a lower and upper index. It calls the partition function and records the returned index of where the pivot ends up. It then calls itself recursively twice; once on the "left" with the lower and pivot-1 indices, and once on the right with the pivit+1 and upper indices.<br><br>
The only part of the algorithm that can be improved is the selection of the pivot. A good one gives good performance while a bad one can give $O(n^{2})$ complexity. In almost any situation, you have to <I>intentionally try</I> to actually run into the worst-case (unless you're sorting a pre-sorted array), so an average case that's acceptable seems to almost always appear in practice.

In [2]:
def partition(A, low, high, pivot=None):
    """Sorts list around pivot such that all values to the left
    are smaller and all to the right are larger. Inserts pivot in between
    partition before returning index of pivot.
    """
    if pivot == None:   #If no pivot is specified
        pivot = A[high] #set pivot to last item in array
    else:
        try:
            high = A.index(pivot) #get index for pivot
        except:
            pivot = A[high] #if value not in list, initialize as with no pivot
            
    first = low-1  #first is marker on left hand side
    
    for j in range(low, high): #iterate over every element in list, left to right
        if A[j] <= pivot: #if element is lower or equal to pivot (belongs on left)...
            
            first += 1     #iterate lower index marker
            A[first], A[j] = A[j], A[first]  #swap with value at first to place left of pivot
            
    A[first+1], A[high] = A[high], A[first+1] #insert pivot between partition
    return first + 1 #return position of pivot

def quicksort(A, low, high):
    """quicksort! applies partition function to array and itself recursively to
    both sides of partition.
    """
    if high - low <= 1:   #End condition where subpartition length is 1 or 0.
        return
    sep = partition(A, low, high)  #call partition on array at indices, record pivot pos
    quicksort(A, low, sep-1)   #recurse at left partition
    quicksort(A, sep+1, high)   #recurse at right partition
    return
    

## Sanity Check: Does it work?
Run the algorithm on the short list and verify it's working.

In [3]:
lst = short_list.copy()
print(lst)
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')
print(lst)

[152745, 481850, 477025, 997948, 808225, 183236, 739784, 412125, 767514, 366725]
sorted in:  0.0004208087921142578  seconds.
[152745, 183236, 366725, 412125, 477025, 481850, 739784, 767514, 808225, 997948]


Looks like it worked. Now for the big check: how does it do on the long list?

In [4]:
lst = long_list.copy()
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.06067252159118652  seconds.


Hey, not bad. Let's finally see if what they say is true: that sorting a pre-sorted list takes a long time.

In [5]:
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

RecursionError: maximum recursion depth exceeded in comparison

Maximum recursion depth exceeded? Bah! I must increase the recursion depth, then!

In [6]:
import sys
sys.setrecursionlimit(len(long_list))  #set recursion limit to length of long list.

And we'll see if it breaks my computer!

In [7]:
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  15.055406332015991  seconds.


Sure enough, it does a really bad job on a pre-sorted list. Funny the way that works.